In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

Wczytanie danych

In [13]:
df = pd.read_csv('kaggle/Diabetes Classification.csv')

Sprawdź brakujące wartości

In [14]:
print(df.isnull().sum())

Unnamed: 0    0
Age           0
Gender        0
BMI           0
Chol          0
TG            0
HDL           0
LDL           0
Cr            0
BUN           0
Diagnosis     0
dtype: int64


Podział danych na cechy (X) i etykiety (y)

In [15]:
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

Podział danych na zbiór treningowy i testowy

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)